In [ ]:
#| default_exp evaluation

# Datasets Evaluation

In [ ]:
#| export
import pickle
from functools import partial
from typing import Any, Callable, Dict, List, Optional, Union

import numpy as np
import pandas as pd
from fugue import transform
from fugue.collections.yielded import Yielded
from fugue.constants import FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT
from fugue.dataframe import DataFrame
from fugue.workflow import FugueWorkflow

In [ ]:
#| export
def _cotransform(
    df1: Any,
    df2: Any,
    using: Any,
    schema: Any = None,
    params: Any = None,
    partition: Any = None,
    engine: Any = None,
    engine_conf: Any = None,
    force_output_fugue_dataframe: bool = False,
    as_local: bool = False,
) -> Any:
    dag = FugueWorkflow(compile_conf={FUGUE_CONF_WORKFLOW_EXCEPTION_INJECT: 0})
    src = dag.create_data(df1).zip(dag.create_data(df2), partition=partition)
    tdf = src.transform(
        using=using,
        schema=schema,
        params=params,
        pre_partition=partition,
    )
    tdf.yield_dataframe_as("result", as_local=as_local)
    dag.run(engine, conf=engine_conf)
    result = dag.yields["result"].result  # type:ignore
    if force_output_fugue_dataframe or isinstance(df1, (DataFrame, Yielded)):
        return result
    return result.as_pandas() if result.is_local else result.native

In [ ]:
#| export
def _evaluate(
        df: pd.DataFrame, 
        df_train: pd.DataFrame,
        metrics: Optional[List[Callable]] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
    ) -> pd.DataFrame:
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    y = df[target_col].values
    eval_ = {}
    for model in models:
        eval_[model] = {}
        for metric in metrics:
            y_hat = df[model].values
            metric_name = metric.__name__
            if 'mase' in metric_name:
                if df_train is None:
                    raise Exception('Please provide `Y_df` to compute mase')
                metric_res = metric(y, y_hat, df_train[target_col].values)
            else:
                metric_res = metric(y, y_hat)
            eval_[model][metric_name] = metric_res
    eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
    if has_cutoff:
        eval_df.insert(0, 'cutoff', df['cutoff'].iloc[0])
    eval_df.insert(0, id_col, df[id_col].iloc[0])
    return eval_df

In [ ]:
#| export
def _evaluate_without_insample(
        df: pd.DataFrame, 
        metrics: Optional[List[Callable]] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
    ) -> pd.DataFrame:
    return _evaluate(
        df=df, df_train=None, metrics=metrics, id_col=id_col, time_col=time_col,
        target_col=target_col, level=level,
    )

In [ ]:
#| export
def _schema_evaluate(
        df: pd.DataFrame,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ):
    cols_to_rm = '|'.join([id_col, time_col, target_col, 'cutoff', 'lo', 'hi'])
    has_cutoff = 'cutoff' in df.columns
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    str_models = ','.join([f"{model}:double" for model in models])
    dtypes = df.dtypes
    id_col_type = dtypes.loc[id_col]
    if id_col_type == 'category':
        raise NotImplementedError(
            'Use of `category` type to identify each time series is not yet implemented. '
            f'Please transform your {id_col} to string to continue.'
        )
    id_col_type = 'string' if id_col_type == 'object' else id_col_type
    cutoff_col_type = ''
    if has_cutoff:
        cutoff_col_type = f"{dtypes.loc['cutoff']}".replace('64[ns]', '')
        cutoff_col_type = 'string' if cutoff_col_type == 'object' else cutoff_col_type
    schema = (
        f'{id_col}:{id_col_type},metric:string,'
        + (f'cutoff:{cutoff_col_type},' if has_cutoff else '')
        + str_models
    )
    return schema

In [ ]:
#| export
def _agg_evaluation(
        df_eval: pd.DataFrame, 
        agg_fn: Any, 
        agg_by: Any,
    ) -> pd.DataFrame:
    cols_to_rm = '|'.join(agg_by + ['unique_id', 'metric', 'cutoff', 'lo', 'hi'])
    models = df_eval.loc[:, ~df_eval.columns.str.contains(cols_to_rm)].columns
    return df_eval.groupby(agg_by)[models].apply(agg_fn, axis=0).reset_index()

In [ ]:
#| export
def _schema_agg_evaluation(
        df: pd.DataFrame, 
        agg_by: Optional[List[str]] = None,
    ) -> pd.DataFrame:
    cols_to_rm = '|'.join(agg_by + ['unique_id', 'metric', 'cutoff', 'lo', 'hi'])
    models = df.loc[:, ~df.columns.str.contains(cols_to_rm)].columns
    str_models = ','.join([f'{model}:double' for model in models])
    dtypes = df.dtypes
    agg_by_types = [dtypes.loc[col] for col in agg_by]
    if 'category' in agg_by_types:
        raise NotImplementedError(
            'Use of `category` type is not yet implemented. '
            f'Please transform your columns to string to continue.'
        )
    agg_by_types = ['string' if col == 'object' else col for col in agg_by_types]
    agg_by_types = [col.replace('64[ns]', '') for col in agg_by_types]
    schema = [f'{col}:{type_}' for col, type_ in zip(agg_by, agg_by_types)]
    schema = ','.join(schema) + ',' + str_models
    return schema

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail, test_close
from datasetsforecast.m4 import M4, M4Evaluation

In [ ]:
#| hide
fforma_url = 'https://github.com/Nixtla/m4-forecasts/raw/master/forecasts/submission-245.zip'
M4Evaluation.load_benchmark('data', 'Hourly', fforma_url)
eval_test = pd.read_csv('data/m4/datasets/submission-245.csv').query("id.str.startswith('H')")
eval_test = eval_test.set_index('id')
eval_test.columns = list(range(1, 49))
eval_test = eval_test.unstack().reset_index()
eval_test.columns = ['ds', 'unique_id', 'FFORMA']
eval_test = eval_test[['unique_id', 'ds', 'FFORMA']].sort_values(['unique_id', 'ds'])
hourly, *_ = M4.load('data', 'Hourly')
test_hourly = hourly.groupby('unique_id').tail(48)
train_hourly = hourly.drop(test_hourly.index)
test_hourly['FFORMA'] = eval_test['FFORMA'].values

/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_44075/3530693183.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_hourly['FFORMA'] = eval_test['FFORMA'].values


In [ ]:
#| hide
test_eq(
    _schema_evaluate(eval_test),
    'unique_id:string,metric:string,FFORMA:double'
)

In [ ]:
#| export
def evaluate_forecast(
        Y_hat_df: pd.DataFrame,
        metrics: List[Callable],
        Y_test_df: Optional[pd.DataFrame] = None,
        Y_df: Optional[pd.DataFrame] = None,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        level: Optional[List] = None,
        agg_fn: Callable = np.mean,
        agg_by: Optional[List[str]] = None,
        engine: Any = None,
        **transform_kwargs: Any,
    ) -> pd.DataFrame:
    if 'y' not in Y_hat_df.columns:
        raise Exception(
            'Please include the actual values in `Y_hat_df` '
            'or pass `Y_test_df`.'
        )
    df = Y_hat_df if Y_test_df is None else Y_hat_df.merge(Y_test_df, how='left', on=[id_col, time_col])
    transform_fn = partial(_cotransform, df2=Y_df) if Y_df is not None else transform   
    if Y_df is None:
        fn = _evaluate_without_insample
    else:
        fn = _evaluate
    has_cutoff = 'cutoff' in Y_hat_df.columns
    evaluation_df = transform_fn(
        df, 
        using=fn, 
        engine=engine, 
        params=dict(
            metrics=metrics,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        ), 
        schema=_schema_evaluate(
            df, 
            id_col=id_col, 
            time_col=time_col, 
            target_col=target_col,
        ), 
        partition=dict(by=id_col) if not has_cutoff else dict(by=[id_col, 'cutoff']),
    )
    if agg_by is not None:
        agg_by = ['metric'] + agg_by
    else:
        agg_by = ['metric']
    evaluation_df = transform(
        evaluation_df,
        using=_agg_evaluation,
        engine=engine,
        params=dict(agg_fn=agg_fn, agg_by=agg_by),
        schema=_schema_agg_evaluation(evaluation_df, agg_by),
        partition=agg_by,
        **transform_kwargs,
    )
    return evaluation_df

In [ ]:
#| hide
from datasetsforecast.losses import smape, mase, mape

In [ ]:
#| hide
def mase_24(y, y_hat, y_train):
    return mase(y, y_hat, y_train, seasonality=24)

In [ ]:
#| hide
test_cutoff = pd.concat([test_hourly.assign(cutoff=str(i), model1=0) for i in range(3)])
eval_cutoff = evaluate_forecast(test_cutoff, metrics=[smape, mape], agg_by=['cutoff'])
test_eq(
    eval_cutoff.shape,
    (6, 4)
)

In [ ]:
#| hide
test_close(
    evaluate_forecast(test_hourly, Y_df=train_hourly, metrics=[smape, mase_24])['FFORMA'].values,
    np.array([0.818598, 11.505702])
)

In [ ]:
#| hide
test_eq(
    evaluate_forecast(test_hourly, Y_df=train_hourly, metrics=[smape]),
    evaluate_forecast(test_hourly, metrics=[smape])
)

In [ ]:
#| hide
test_fail(
    lambda : evaluate_forecast(test_hourly, metrics=[smape, mase]), 
    contains='mase'
)

ERROR:root:_1 _State.RUNNING -> _State.FAILED  Please provide `Y_df` to compute mase
